In [1]:
import pandas as pd
import numpy as np
import sqlalchemy
import json
from sqlalchemy import create_engine, MetaData
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, String, Numeric, Text, Float, ForeignKey
from sqlalchemy.orm import sessionmaker, relationship

from flask import Flask, jsonify, render_template
from flask_sqlalchemy import SQLAlchemy

In [2]:
# Create Engine
engine = create_engine("sqlite:///housingdata.sqlite")
# Declare a Base
Base = declarative_base()

In [3]:
# Define the ORM class for `Measurements`
class Measurement(Base):
    
    __tablename__ = 'hdata'

    id = Column(Integer, primary_key=True)
    QUADRANT = Column(String)
    FULLADDRESS = Column(String)
    ZIPCODE = Column(Integer)
    BEDRM = Column(Integer)
    BATHRM = Column(Integer)
    KITCHENS = Column(Integer)
    PRICE = Column(Float)

In [5]:
# Use `create_all` to create the tables
Base.metadata.create_all(engine)

In [6]:
# Use Pandas to Bulk insert each CSV file into their appropriate table
def populate_table(engine, table, csvfile):
    """Populates a table from a Pandas DataFrame."""
    # connect to the database
    conn = engine.connect()
    
    # Load the CSV file into a pandas dataframe 
    df_of_data_to_insert = pd.read_csv(csvfile)
    
    # Orient='records' creates a list of data to write
    # http://pandas-docs.github.io/pandas-docs-travis/io.html#orient-options
    data = df_of_data_to_insert.to_dict(orient='records')

    # Optional: Delete all rows in the table 
    conn.execute(table.delete())

    # Insert the dataframe into the database in one bulk insert
    conn.execute(table.insert(), data)

In [7]:
populate_table(engine, Measurement.__table__, 'hdtable.csv')

In [8]:
engine.execute("SELECT * FROM hdata LIMIT 10").fetchall()

[(1, 'NW', '1416 CORCORAN STREET NW', 20009, 4, 3, 2, 1700000.0),
 (2, 'NW', '1910 1/2 NEW HAMPSHIRE AVENUE NW', 20009, 3, 2, 2, 1363000.0),
 (3, 'NW', '1619 SWANN STREET NW', 20009, 4, 6, 4, 1500000.0),
 (4, 'NW', '1707 21ST STREET NW', 20009, 6, 4, 4, 1351830.0),
 (5, 'NW', '2025 R STREET NW', 20009, 6, 4, 2, 1844970.0),
 (6, 'NW', '2014 HILLYER PLACE NW', 20009, 5, 3, 2, 1625000.0),
 (7, 'NW', '1760 T STREET NW', 20009, 5, 2, 2, 1925000.0),
 (8, 'NW', '1808 CORCORAN STREET NW', 20009, 3, 2, 1, 950000.0),
 (9, 'NW', '1639 19TH STREET NW', 20009, 3, 2, 2, 1330000.0),
 (10, 'NW', '1524 18TH STREET NW', 20036, 4, 4, 4, 2279000.0)]